In [2]:
import re
import json
import stanza

import numpy as np
import pandas as pd

from itertools import chain
from ast import literal_eval

from preprocessing_functions import load_txt_as_lst, split_txt, \
run_stanza, flatten_list, segment_series, preprocess_series, write_file

# Lucretius en1893 no section markers

In [3]:
raw_str_path = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1893_str.txt"
en1893_raw_lst = load_txt_as_lst(raw_str_path)

In [4]:
en1893_raw_lst[:10]

['REMARKS\n',
 '\n',
 'ON THE\n',
 '   LIFE AND POEM OF LUCRETIUS.\n',
 '  OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.\n',
 '\n',
 'That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2\n',
 '\n',
 'As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquated character may be partly affected, in imitation,  perhaps, of Ennius, for whom he expresses great veneration.3\n',
 '\n']

In [5]:
en1893_str = " ".join(en1893_raw_lst)

In [6]:
# load stanza model for lang
lang_ = "en"
stanza_model_ = stanza.Pipeline(lang=lang_, processors='tokenize', use_gpu=True)

2023-06-22 10:40:43 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-06-22 10:40:43 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2023-06-22 10:40:43 WARNING: GPU requested, but is not available!
2023-06-22 10:40:43 INFO: Using device: cpu
2023-06-22 10:40:43 INFO: Loading: tokenize
2023-06-22 10:40:43 INFO: Done loading processors!


In [7]:
en1983_sents = preprocess_series(en1893_str, "en", stanza_model_)

segmented str into sentences


In [8]:
len(en1983_sents)

13966

In [9]:
en1983_sents[:10]

['REMARKS',
 'ON THE',
 'LIFE AND POEM OF LUCRETIUS.',
 'OF the life of Lucretius but little information has reached us.',
 'Ad nos vix tenuis famæ perlabitur aura.',
 'That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.',
 '2',
 'As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.',
 'At this period, Ennius had been dead about seventy years ;',
 'Cicero was in his twelfth year;']

In [10]:
# path_out = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1893_sents.txt"
# write_file(en1983_sents, path_out)

# Lucretius en1893 with section markers

In [28]:
raw_path_markers = "/home/craig.car/repos/chiron/align_texts_project/data/lucretius/lucretius_en1893_str_withsections.txt"
en1893_raw_lst_markers = load_txt_as_lst(raw_path_markers)

In [29]:
en1893_raw_lst_markers[:10]

['#@$%title#@$%\n',
 'REMARKS\n',
 '#@$%paragraph end#@$%\n',
 '\n',
 '#@$%title#@$%\n',
 'ON THE\n',
 '   #@$%title#@$%\n',
 'LIFE AND POEM OF LUCRETIUS.\n',
 '  OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.\n',
 '#@$%paragraph end#@$%\n']

In [30]:
en1893_str_markers = " ".join(en1893_raw_lst_markers)

In [31]:
en1893_str_markers[:1000]

'#@$%title#@$%\n REMARKS\n #@$%paragraph end#@$%\n \n #@$%title#@$%\n ON THE\n    #@$%title#@$%\n LIFE AND POEM OF LUCRETIUS.\n   OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura.\n #@$%paragraph end#@$%\n \n That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2\n #@$%paragraph end#@$%\n \n As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquated character may be partly affected, in imitation,  perhaps, of Ennius, for whom 

In [32]:
en1893_str_markers = en1893_str_markers.replace("\n", "")

In [33]:
en1893_str_markers[:1000]

'#@$%title#@$% REMARKS #@$%paragraph end#@$%  #@$%title#@$% ON THE    #@$%title#@$% LIFE AND POEM OF LUCRETIUS.   OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura. #@$%paragraph end#@$%  That he was a Roman by birth, is inferred from the pas  sages in his poem in which he speaks of the Roman world as  his country,1 and of the Roman language as his native tongue.2 #@$%paragraph end#@$%  As to the time of his birth, it is stated by Eusebius in his  Chronicon, that he was born in the second year of the hundred  and seventy-first Olympiad, or ninety-five years before Christ.  At this period, Ennius had been dead about seventy years ;  Cicero was in his twelfth year; twenty-five years were to  elapse before the birth of Virgil, and four before that of Julius  Caesar. His style, indeed, would make him seem older, but  its antiquated character may be partly affected, in imitation,  perhaps, of Ennius, for whom he expresses great veneratio

## Split by section (including chapters)
Note: "#@$%" added at beginning and end of section markers when extracting text from xml

In [34]:
en1893_str_markers_split = en1893_str_markers.split("#@$%")

In [35]:
en1893_str_markers_split[:10]

['',
 'title',
 ' REMARKS ',
 'paragraph end',
 '  ',
 'title',
 ' ON THE    ',
 'title',
 ' LIFE AND POEM OF LUCRETIUS.   OF the life of Lucretius but little information has reached us.  Ad nos vix tenuis famæ perlabitur aura. ',
 'paragraph end']

In [36]:
# define patterns
title_marker = re.compile("title")
title_commentary = re.compile("ARGUMENT")
title_book = re.compile("BOOK")
title_index = re.compile("INDEX")
para_end = re.compile("paragraph end")
note_start = re.compile("note")
note_end = re.compile("note_end")